In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [17]:
j_faults = pd.read_csv('data/J1939Faults.csv').reset_index(drop = True)
onboard_data = pd.read_csv('data/VehicleDiagnosticOnboardData.csv').reset_index(drop=True)
fault_codes = pd.read_csv('data/Service_Fault_Codes_1_0_0_167.csv').reset_index(drop=True)
vehicle = pd.read_csv('data/Vehicle_Make.csv').reset_index(drop=True)

In [18]:
j_faults.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000


In [45]:
j_faults[j_faults['spn'] == 5246]

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
45,46,990931,2015-02-21 12:10:51.000,NaN,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,True,1,NaN,1395,105349612,36.065972,-86.433425,2015-02-21 12:11:27.000
1918,1919,1007751,2015-02-22 19:44:55.000,NaN,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,True,1,NaN,1395,105349612,36.066203,-86.434814,2015-02-22 19:46:27.000
2058,2059,1010486,2015-02-23 04:00:21.000,NaN,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,False,1,NaN,1395,105349612,36.066666,-86.434537,2015-02-23 01:06:06.000
2089,2090,1011009,2015-02-23 05:05:44.000,NaN,NaN,05290170*03015749*051914190353*09400015*G1*BDR*,79642446,6X1u13D1500000000,CMMNS,0,5246,0,True,1,NaN,1630,105329900,40.733009,-74.087777,2015-02-23 05:08:23.000
2971,2972,1026305,2015-02-23 15:54:22.000,NaN,NaN,unknown,unknown,unknown,unknown,0,5246,0,True,1,NaN,1487,105369355,28.077361,-81.897083,2015-02-23 15:54:58.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183032,1244156,121610128,2020-02-19 07:02:33.000,NaN,NaN,05317106*05005224*051718172255*09401583*G1*BDR*,79845785,6X1u13D1500000000,CMMNS,0,5246,0,True,1,NaN,1814,105369518,36.067037,-86.434120,2020-02-19 07:03:09.000
1183684,1244808,121909497,2020-02-21 07:23:44.000,NaN,NaN,04384413*22246857*090619141107*60701756*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,5246,16,True,1,NaN,2211,105329862,36.066296,-86.434305,2020-02-21 07:24:20.000
1184328,1245452,122305094,2020-02-24 15:28:05.000,NaN,NaN,04384413*22246857*090619141107*60701756*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,5246,16,False,1,NaN,2211,105329862,36.066620,-86.434722,2020-02-24 15:28:01.000
1184330,1245454,122305096,2020-02-24 15:27:26.000,NaN,NaN,04384413*22246857*090619141107*60701756*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,5246,0,True,1,NaN,2211,105329862,36.066620,-86.434722,2020-02-24 15:28:02.000


In [19]:
onboard_data.head()

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1


In [43]:
ob_data = onboard_data.pivot(index='FaultId', columns='Name')
ob_data

Id                                         \
Name    AcceleratorPedal BarometricPressure CruiseControlActive   
FaultId                                                           
1                    6.0               12.0                17.0   
2                    NaN                NaN                 NaN   
3                    NaN                NaN                 NaN   
4                    NaN                NaN                 NaN   
5                    NaN                NaN                 NaN   
...                  ...                ...                 ...   
1248454              NaN                NaN                 NaN   
1248455       12863965.0         12863973.0          12863977.0   
1248456       12863987.0         12863994.0          12863998.0   
1248457       12864007.0         12864014.0          12864019.0   
1248458              NaN                NaN                 NaN   

                                                                    \
Name    CruiseControlSetSpeed DistanceLtd EngineCoolantTemperature   
FaultId                                                              
1                        16.0        21.0                     18.0   
2                         NaN         NaN                      NaN   
3                         NaN         NaN                      NaN   
4                         NaN         NaN                      NaN   
5                         NaN         NaN                      NaN   
...                       ...         ...                      ...   
1248454                   NaN         NaN                      NaN   
1248455            12863978.0  12863981.0               12863979.0   
1248456            12863999.0  12864001.0               12864000.0   
1248457            12864018.0  12864023.0               12864020.0   
1248458                   NaN         NaN                      NaN   

                                                                        ...  \
Name     EngineLoad EngineOilPressure EngineOilTemperature   EngineRpm  ...   
FaultId                                                                 ...   
1               5.0               2.0                  3.0        10.0  ...   
2               NaN               NaN                  NaN         NaN  ...   
3               NaN               NaN                  NaN         NaN  ...   
4               NaN               NaN                  NaN         NaN  ...   
5               NaN               NaN                  NaN         NaN  ...   
...             ...               ...                  ...         ...  ...   
1248454         NaN               NaN                  NaN         NaN  ...   
1248455  12863967.0        12863962.0           12863963.0  12863971.0  ...   
1248456  12863986.0        12863984.0           12863985.0  12863992.0  ...   
1248457  12864008.0        12864004.0           12864005.0  12864012.0  ...   
1248458         NaN               NaN                  NaN         NaN  ...   

                  Value                                                 \
Name    FuelTemperature IgnStatus IntakeManifoldTemperature LampStatus   
FaultId                                                                  
1                   NaN     False                      78.8       1023   
2                   NaN      True                       NaN       1279   
3                   NaN       NaN                       NaN       1279   
4                   NaN      True                       NaN       1279   
5                   NaN       NaN                       NaN      16639   
...                 ...       ...                       ...        ...   
1248454             NaN       NaN                       NaN       1023   
1248455              32      True                      98.6      18431   
1248456             NaN      True                      91.4      17407   
1248457             NaN      True                     100.4       1023   
1248458             NaN       NaN

In [44]:
pd.set_option('display.max_columns', None)
ob_data

Id                                         \
Name    AcceleratorPedal BarometricPressure CruiseControlActive   
FaultId                                                           
1                    6.0               12.0                17.0   
2                    NaN                NaN                 NaN   
3                    NaN                NaN                 NaN   
4                    NaN                NaN                 NaN   
5                    NaN                NaN                 NaN   
...                  ...                ...                 ...   
1248454              NaN                NaN                 NaN   
1248455       12863965.0         12863973.0          12863977.0   
1248456       12863987.0         12863994.0          12863998.0   
1248457       12864007.0         12864014.0          12864019.0   
1248458              NaN                NaN                 NaN   

                                                                    \
Name    CruiseControlSetSpeed DistanceLtd EngineCoolantTemperature   
FaultId                                                              
1                        16.0        21.0                     18.0   
2                         NaN         NaN                      NaN   
3                         NaN         NaN                      NaN   
4                         NaN         NaN                      NaN   
5                         NaN         NaN                      NaN   
...                       ...         ...                      ...   
1248454                   NaN         NaN                      NaN   
1248455            12863978.0  12863981.0               12863979.0   
1248456            12863999.0  12864001.0               12864000.0   
1248457            12864018.0  12864023.0               12864020.0   
1248458                   NaN         NaN                      NaN   

                                                                        \
Name     EngineLoad EngineOilPressure EngineOilTemperature   EngineRpm   
FaultId                                                                  
1               5.0               2.0                  3.0        10.0   
2               NaN               NaN                  NaN         NaN   
3               NaN               NaN                  NaN         NaN   
4               NaN               NaN                  NaN         NaN   
5               NaN               NaN                  NaN         NaN   
...             ...               ...                  ...         ...   
1248454         NaN               NaN                  NaN         NaN   
1248455  12863967.0        12863962.0           12863963.0  12863971.0   
1248456  12863986.0        12863984.0           12863985.0  12863992.0   
1248457  12864008.0        12864004.0           12864005.0  12864012.0   
1248458         NaN               NaN                  NaN         NaN   

                                                                           \
Name    EngineTimeLtd   FuelLevel     FuelLtd    FuelRate FuelTemperature   
FaultId                                                                     
1                15.0        13.0         9.0         8.0             NaN   
2                 NaN         NaN         NaN         NaN             NaN   
3                 NaN         NaN         NaN         NaN             NaN   
4                 NaN         NaN         NaN         NaN             NaN   
5                 NaN         NaN         NaN         NaN             NaN   
...               ...         ...         ...         ...             ...   
1248454           NaN         NaN         NaN         NaN             NaN   
1248455    12863976.0  12863974.0  12863970.0  12863969.0      12863964.0   
1248456    12863997.0  12863995.0  12863989.0  12863991.0             NaN   
1248457    12864017.0  12864015.0  12864011.0  12864010.0             NaN   
1248458           NaN         NaN         NaN         NaN       

In [38]:
onboard_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12821626 entries, 0 to 12821625
Data columns (total 4 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   Id       int64 
 1   Name     object
 2   Value    object
 3   FaultId  int64 
dtypes: int64(2), object(2)
memory usage: 391.3+ MB


In [ ]:
pvt

In [46]:
fault_codes

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
0,Y,111,167,Not Mapped,254,0,12,629,12,P0606,Red,Stop / Shutdown,Engine Control Module Critical Internal Failur...,Error internal to the ECM related to memory ha...
1,Y,112,167,Not Mapped,20,128,7,635,7,Not Mapped,Red,Stop / Shutdown,Engine Timing Actuator Driver Circuit - Mechan...,Mechanical failure in the engine timing actuat...
2,Y,113,167,Not Mapped,20,128,3,635,3,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,High signal voltage detected at the engine tim...
3,Y,114,167,Not Mapped,20,128,4,635,4,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,Low voltage detected at the engine timing actu...
4,Y,115,167,190,Not Mapped,Not Mapped,2,612,2,P0008,Red,Stop / Shutdown,Engine Magnetic Speed/Position Lost Both of Tw...,The ECM has detected that the primary and back...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7119,Y,9996,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7120,Y,9997,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7121,Y,9998,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7122,Y,9999,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN


In [21]:
vehicle.head()

,Truck Number,Year,Make,Serial
0,535,1996,PETERBILT,1XPCDR8XXVN431011
1,534,1996,PETERBILT,1XPCPR8X8VN431010
2,533,1996,PETERBILT,1XPCDR8X1VN431009
3,532,1996,PETERBILT,1XPCDR8XXVN431008
4,531,1996,PETERBILT,1XPCDR8X8VN431007


In [54]:
ob_work = ob_data[0:100000]
faults_work = j_faults[j_faults['RecordID']<100000]
faults_work.head()
#ob_faults = pd.merge(faults_work, ob_work, how='left', on=['RecordID','FaultId'])

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000


In [55]:
ob_faults = pd.merge(faults_work, ob_work, how='left', left_on=['RecordID'], right_on=['FaultId'])

C:\Users\cmerr\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [56]:
ob_faults

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,"(Id, AcceleratorPedal)","(Id, BarometricPressure)","(Id, CruiseControlActive)","(Id, CruiseControlSetSpeed)","(Id, DistanceLtd)","(Id, EngineCoolantTemperature)","(Id, EngineLoad)","(Id, EngineOilPressure)","(Id, EngineOilTemperature)","(Id, EngineRpm)","(Id, EngineTimeLtd)","(Id, FuelLevel)","(Id, FuelLtd)","(Id, FuelRate)","(Id, FuelTemperature)","(Id, IgnStatus)","(Id, IntakeManifoldTemperature)","(Id, LampStatus)","(Id, ParkingBrake)","(Id, ServiceDistance)","(Id, Speed)","(Id, SwitchedBatteryVoltage)","(Id, Throttle)","(Id, TurboBoostPressure)","(Value, AcceleratorPedal)","(Value, BarometricPressure)","(Value, CruiseControlActive)","(Value, CruiseControlSetSpeed)","(Value, DistanceLtd)","(Value, EngineCoolantTemperature)","(Value, EngineLoad)","(Value, EngineOilPressure)","(Value, EngineOilTemperature)","(Value, EngineRpm)","(Value, EngineTimeLtd)","(Value, FuelLevel)","(Value, FuelLtd)","(Value, FuelRate)","(Value, FuelTemperature)","(Value, IgnStatus)","(Value, IntakeManifoldTemperature)","(Value, LampStatus)","(Value, ParkingBrake)","(Value, ServiceDistance)","(Value, Speed)","(Value, SwitchedBatteryVoltage)","(Value, Throttle)","(Value, TurboBoostPressure)"
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000,6.0,12.0,17.0,16.0,21.0,18.0,5.0,2.0,3.0,10.0,15.0,13.0,9.0,8.0,NaN,1.0,7.0,11.0,19.0,NaN,14.0,20.0,NaN,4.0,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,0,1632.2,43.2,12300.907429328,0,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,NaN,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97609,99995,3161626,2015-06-26 07:00:06.000,Abnormal Update Rate Tire Location,NaN,unknown,unknown,unknown,unknown,49,929,9,True,126,NaN,1641,105420520,29.992268,-82.584675,2015-06-26 07:00:42.000,981383.0,981390.0,981393.0,981394.0,981398.0,981395.0,981382.0,981379.0,981380.0,981389.0,981392.0,NaN,981387.0,981386.0,981381.0,981378.0,981385.0,981388.0,98